In [2]:
import os
import seaborn as sns
path = '/Users/connormcdonald/Desktop/Masters/MIT807/Gartner Repository/Analysis/Figures'
import sys
sys.path.insert(1, '/Users/connormcdonald/Desktop/Masters/MIT807/Gartner Repository/Data Collection')
from configs import *
import numpy as np
import statsmodels.formula.api as smf
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from mpl_toolkits.axisartist.axislines import SubplotZero
from pylab import text
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "lualatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

from sqlalchemy import create_engine

engine = create_engine('mysql+mysqlconnector://'+user+':'+passwd+'@'+ip+':3306/'+schema1)

In [2]:
stmt = ''' SELECT 
DATE_FORMAT(A.date, \'%Y-%m\') as date, 
COUNT(1) as num_tweets,
AVG(B.mnb_prediction) AS mnb,
AVG(B.svm_prediction) AS svm,
AVG(B.lr_prediction) AS lr,
AVG(B.ensemble_prediction) AS ens
FROM social.five_g_only A
INNER JOIN social.fiveG_sentiment B 
ON A.id = B.id
GROUP BY DATE_FORMAT(A.date, \'%Y-%m\')
ORDER BY DATE_FORMAT(A.date, \'%Y-%m\') ASC'''

df = pd.read_sql(stmt, con=engine)
df['date'] = pd.to_datetime(df['date'])

In [3]:
ax = plt.gca()
line = pd.to_datetime('2017-06-01')
line2 = pd.to_datetime('2018-09-01')

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.rcParams["figure.figsize"] = [8, 5]
plt.rcParams.update({'font.size': 12})
plt.rcParams['figure.dpi'] = 300
ax.set_ylim([0,1])
plt.xlabel('Time')
plt.ylabel('Proportion of Positive Tweets')
plt.plot(df.date, df.mnb, label = 'MNB')
plt.plot(df.date, df.lr, label = 'LR')
plt.plot(df.date, df.svm, label = 'SVM')
plt.plot(df.date, df.ens, label = 'Ensemble')
plt.axvline(x=line, c = '#999999', linewidth = 1, linestyle ="--", label = 'PoI')
plt.axvline(x=line2, c = '#999999', linewidth = 1, linestyle ="--")
plt.legend(loc='lower right')
plt.savefig(os.path.join(path, '5g_sentiment.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
plt.close()

In [3]:
stmt = ''' SELECT 
DATE_FORMAT(A.date, \'%Y-%m\') as date, 
AVG(B.mnb_probability) AS mnb_probability,
AVG(B.svm_probability) AS svm_probability,
AVG(B.lr_probability) AS lr_probability
FROM social.five_g_only A
INNER JOIN social.fiveG_sentiment B 
ON A.id = B.id
GROUP BY DATE_FORMAT(A.date, \'%Y-%m\')
ORDER BY DATE_FORMAT(A.date, \'%Y-%m\') ASC'''

df = pd.read_sql(stmt, con=engine)
df['date'] = pd.to_datetime(df['date'])

In [4]:
df['ens_probability'] = (df['mnb_probability'] + df['svm_probability'] + df['lr_probability'])/3

In [17]:
ax = plt.gca()
line = pd.to_datetime('2017-06-01')
line2 = pd.to_datetime('2018-09-01')

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.rcParams["figure.figsize"] = [8, 4.5]
plt.rcParams.update({'font.size': 12})
plt.rcParams['figure.dpi'] = 300
ax.set_ylim([0,1])
plt.xlabel('Time')
plt.ylabel('Prediction Probability')
plt.plot(df.date, df.mnb_probability, label = 'MNB Probability')
plt.plot(df.date, df.lr_probability, label = 'LR Probability')
plt.plot(df.date, df.svm_probability, label = 'SVM Probability')
plt.plot(df.date, df.ens_probability, label = 'Ensemble Probability')
plt.axvline(x=line, c = '#999999', linewidth = 1, linestyle ="--", label = 'PoI')
plt.axvline(x=line2, c = '#999999', linewidth = 1, linestyle ="--")
plt.legend(loc='lower right')
plt.savefig(os.path.join(path, '5g_sentiment_prob.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
plt.close()

In [4]:
stmt = '''  SELECT 
A.text
FROM social.five_g_only A
INNER JOIN social.fiveG_sentiment B 
ON A.id = B.id
WHERE ensemble_prediction = 0'''

negative_df = pd.read_sql(stmt, con=engine)


In [5]:
stmt = '''  SELECT 
A.text
FROM social.five_g_only A
INNER JOIN social.fiveG_sentiment B 
ON A.id = B.id
WHERE ensemble_prediction = 1'''

positive_df = pd.read_sql(stmt, con=engine)

In [6]:
from collections import Counter

negative_words = Counter(" ".join(negative_df["text"]).split()).most_common(100)

positive_words = Counter(" ".join(positive_df["text"]).split()).most_common(100)

In [7]:
neg_df = pd.DataFrame(negative_words, columns=['neg_word', "neg_word_count"])
pos_df = pd.DataFrame(positive_words, columns=['pos_word', "pos_word_count"])

In [8]:
merged_df= pd.merge(neg_df, pos_df, left_on='neg_word', right_on='pos_word', how='inner')

In [9]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.append('5g')
stop_words.append('g')
stop_words.append('5')

for i in range(len(merged_df.index)):
    if str(merged_df.neg_word[i]) in stop_words:
        merged_df = merged_df.drop(i)

In [10]:
merged_df.reset_index(drop=True)

,neg_word,neg_word_count,pos_word,pos_word_count
0,la,246279,la,502122
1,iphone,194904,iphone,58772
2,de,131005,de,413531
3,5,129638,5,74930
4,4g,126436,4g,144143
5,ipod,125247,ipod,109093
6,en,113542,en,234400
7,el,99422,el,187891
8,que,93985,que,117010
9,network,89792,network,131924


In [11]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

merged_df[['neg_word_count_norm', 'pos_word_count_norm']] = scaler.fit_transform(merged_df[['neg_word_count', 'pos_word_count']])

In [12]:
unmerged_pos = merged_df.iloc[:, [2,3, 5]].sort_values(by=['pos_word_count'], ascending=False)
unmerged_neg = merged_df.iloc[:, [0,1, 4]].sort_values(by=['neg_word_count'], ascending=False)

In [13]:
top_10_pos = unmerged_pos.head(10).pos_word.values

In [14]:
top_10_neg = unmerged_neg.head(10).neg_word.values

In [15]:
top_words = list(top_10_pos)
top_words.extend(x for x in top_10_neg if x not in top_words)

In [16]:
f = []
for i in range(len(merged_df.index)):
    if merged_df.neg_word.values[i] in top_words:
        f.append(i)

In [17]:
final_df = merged_df.iloc[f,:]

In [18]:
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.rcParams["figure.figsize"] = [8, 4]
plt.rcParams.update({'font.size': 12})
plt.rcParams['figure.dpi'] = 300
plt.grid(linestyle='--', linewidth=0.5)
ax = sns.scatterplot(x='neg_word_count_norm',y='pos_word_count_norm',hue = 'neg_word',data = final_df,legend='full',s=50)
ax.legend(bbox_to_anchor=(1.04, 1), loc="upper left")
plt.xlabel('Normalised Count in Negative Tweets')
plt.ylabel('Normalised Count in Positive Tweets')
plt.savefig(os.path.join(path, '5G_word_occurrence.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
plt.close()